In [1]:


import os
import requests
from requests.auth import HTTPBasicAuth
import json
import urllib.request
from tqdm import tqdm

import rasterio

In [2]:
import os

In [12]:
os.listdir("model_ckpt")

['unet_train_100.tf.index', 'unet_train_100.tf.data-00000-of-00001']

In [7]:
os.path.isfile("503-1291.tif")

True

In [6]:
open("503-1291.tif")

FileNotFoundError: [Errno 2] No such file or directory: '503-1291.tif'

In [2]:
# authentication
BASE_URL = 'https://api.planet.com/basemaps/v1/mosaics/'
API_KEY = "8fb5d85cdcfc40f6b4b9d3f44227142b" # [from Yihong]
auth = HTTPBasicAuth(API_KEY, '')

mosaic_id = '56f00cc2-6be4-4315-9603-c75d6afab225' # should probablity convert to args
url = f'https://api.planet.com/basemaps/v1/mosaics/{mosaic_id}/quads'
bbox = '-91.51307900019566, 36.970297999852846, -87.49519900023363, 42.50848099959849' #update bbox and page_size

res = requests.get(url=url, auth=auth, params={'bbox':bbox, '_page_size':99999}) 
out = json.loads(res.text)
out


{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/56f00cc2-6be4-4315-9603-c75d6afab225/quads?_page_size=99999&api_key=8fb5d85cdcfc40f6b4b9d3f44227142b&bbox=-91.51307900019566%2C+36.970297999852846%2C+-87.49519900023363%2C+42.50848099959849'},
 'items': [{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/56f00cc2-6be4-4315-9603-c75d6afab225/quads/503-1291?api_key=8fb5d85cdcfc40f6b4b9d3f44227142b',
    'download': 'https://api.planet.com/basemaps/v1/mosaics/56f00cc2-6be4-4315-9603-c75d6afab225/quads/503-1291/full?api_key=8fb5d85cdcfc40f6b4b9d3f44227142b',
    'items': 'https://api.planet.com/basemaps/v1/mosaics/56f00cc2-6be4-4315-9603-c75d6afab225/quads/503-1291/items?api_key=8fb5d85cdcfc40f6b4b9d3f44227142b',
    'thumbnail': 'https://tiles.planet.com/basemaps/v1/planet-tiles/ps_weekly_sen2_normalized_analytic_subscription_2020-08-10_2020-08-17_mosaic/gmap/11/503/756.png?api_key=8fb5d85cdcfc40f6b4b9d3f44227142b'},
   'bbox': [-91.58203123722404,
    42.42345

In [4]:
## testing on Jihong's container ##
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

connect_str = "DefaultEndpointsProtocol=https;AccountName=yihongponding;AccountKey=hnvkEWfrIeYEyLUFcYYKyjujkSWiWcPdV/mS8O5GJ51iWPAkni7opzaA7klbwIGpGY0JANb5pRjGBe1ekHbX9Q==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client("test")

blob_images = container_client.list_blobs()
current_images = []
for b in blob_images:
    current_images.append(b.name)
    print(b.name)


503-1291.tif
504-1291.tif
505-1291.tif
506-1291.tif
507-1291.tif
508-1291.tif
509-1291.tif
510-1291.tif
511-1291.tif
512-1291.tif


In [ ]:
counter = 0
for i in tqdm(out['items']):
    fn_tif = str(i['id']) + ".tif"
    counter += 1
    if(fn_tif in current_images):
        print(i['id'])
    else:
        print(fn_tif)
        ## download the image to local ##
        urllib.request.urlretrieve(i['_links']['download'], fn_tif)
        ## upload into the blob service ##
        with open(fn_tif, "rb") as data:
            container_client.upload_blob(name=fn_tif, data=data)
            current_images.append(fn_tif)
        os.remove(fn_tif)
    if counter == 10:
        break

  0%|          | 0/978 [00:00<?, ?it/s]

503-1291.tif


  0%|          | 1/978 [00:12<3:16:19, 12.06s/it]

504-1291.tif


  0%|          | 2/978 [00:21<2:50:35, 10.49s/it]

505-1291.tif


  0%|          | 3/978 [00:30<2:36:13,  9.61s/it]

506-1291.tif


In [5]:
blob_client = blob_service_client.get_blob_client(container="test", blob="503-1291.tif")
download_file_path = "503-1291.tif"
with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

In [4]:
import gdal

In [12]:
gdal.Warp('503-1291-warp.tif', '503-1291.tif', dstSRS='EPSG:32616', xRes=3, yRes=3)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7ffa328bff90> >

In [3]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Conv2D, 
    concatenate, 
    Dropout, 
    Input, 
    Reshape,
    BatchNormalization, 
    MaxPooling2D, 
    UpSampling2D, 
    ReLU, 
    Conv2DTranspose
)
# data processing pacakges
import numpy as np
import matplotlib.pyplot as plt

2022-01-03 22:19:01.380362: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())
tf.config.list_physical_devices()

2.4.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True


2022-01-03 22:19:03.530010: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-03 22:19:03.531683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-03 22:19:03.566898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-01-03 22:19:03.566955: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-03 22:19:03.569108: I tensorflow/stream_executor/platform/default/ds

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# unet model
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(input_img, outputs)
    return model

In [6]:
CKPT_DIR = 'unet_train_100.tf'
FILE_DIR = '*warp.tif'
OUT_DIR = './tmp_pred/'
THRESHOLD = 0.5
img_size = 224
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
input_img = Input((img_size, img_size, 5), name='img')
model = get_unet(input_img, n_filters=32, dropout=0.15, batchnorm=True)
model.load_weights(CKPT_DIR)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

2022-01-03 22:19:08.281198: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-01-03 22:19:08.281291: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-03 22:19:08.281329: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-03 22:19:08.281359: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-01-03 22:19:08.281387: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-01-03 22:19:08.281416: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10


In [10]:
def predict_tiff(net, tiff_path, add_gcvi=False):
    base_img = rasterio.open(tiff_path).read()
    base_img = np.transpose(base_img, [1, 2, 0])
    base_img = base_img[:, :, :4]
    if add_gcvi:
        gcvi = base_img[:, :, 3] / base_img[:, :, 1]
        gcvi = gcvi - 1
        gcvi[gcvi > 20] = 20
        gcvi[gcvi < 0] = 0
        gcvi = np.nan_to_num(gcvi, False, 0, 0, 0)
        gcvi = np.expand_dims(gcvi, -1)
        base_img = np.concatenate([base_img, gcvi], 2) 
    print(base_img.shape)
    # track original size and calc how many cuts
    h, w, c = base_img.shape
    h_count = int(h/img_size) + 1
    w_count = int(w/img_size) + 1

    # calculate padded height and width
    h_padded = h_count * img_size
    w_padded = w_count * img_size

    # Pad image and cut into img_size * img_size
    base_img = np.pad(base_img, ((0, h_padded - h), (0, w_padded-w), (0,0)), 'constant')
    base_img = np.reshape(base_img, (h_count, img_size, w_count, img_size, c))
    base_img = np.transpose(base_img, axes=(0, 2, 1, 3, 4))
    base_img = np.reshape(base_img, (-1, img_size, img_size, c)) # nhwc

    # Predict
    out = net.predict(base_img, batch_size = 8)

    # Convert back
    combined = np.zeros((h_padded, w_padded))
    x = 0
    idx = 0
    while x < combined.shape[0]:
        y = 0
        while y < combined.shape[1]:
            combined[x:x+img_size, y:y+img_size] = np.squeeze(out[idx], 2)
            y += img_size
            idx += 1
        x += img_size
    combined_cut = np.array(combined)
    # Trim
    return combined_cut[:h, :w]

In [11]:
 mask = predict_tiff(net = model,
                tiff_path = '503-1291-warp.tif',
                add_gcvi = True)

/home/c3/.conda/envs/py-image/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(5054, 5071, 5)


2021-12-16 22:20:09.611855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-16 22:20:09.613713: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2596985000 Hz
2021-12-16 22:20:10.313336: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-16 22:20:20.120375: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-16 22:20:20.155534: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-16 22:20:21.354930: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [14]:
meta = rasterio.open('503-1291-warp.tif').meta.copy()
meta.update({
    'count':1,
    'dtype':np.uint8
})
mask[mask > 0.5] = 1
mask[mask < 1] = 0
mask = mask.astype(np.uint8)
with rasterio.open("pred.tif", 'w', **meta) as dest:
    dest.write(np.expand_dims(mask, 0))

In [ ]:
urllib.request.urlretrieve(i['_links']['download'], fn_tif)
        ## upload into the blob service ##
with open("pred.tif", "rb") as data:
    container_client.upload_blob(name="pred.tif", data=data)
    current_images.append("pred.tif")
os.remove(fn_tif)
        

In [8]:
technique = c3.KerasTechnique(numEpochs=10, stepsPerEpoch=10, batchSize=32)
keras_pipe = c3.KerasPipe(name="segmentation", 
                          technique=technique).upsertNativeModel(model, upsertModelDefOnly=False)

<string>:988: UserWarning: Local version of tensorflow (2.4.1) is different from the version on C3 AI Suite (2.1.0). This can lead to deserialization issues when the pipe is used on the platform. We recommend to use the same version of the library locally and on the platform.
<string>:2502: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


In [9]:
test = np.random.random((1,224,224,5))
test.shape
X_test = c3.Dataset.fromPython(test)


In [10]:
prediction_classifier = keras_pipe.process(X_test)


500 - NotClassified - c3.love.util.OsUtil_err2 [381.5997]
message: "Error executing command: /usr/local/share/c3/condaEnvs/dti-jupyter/tc01/py-tensorflow_3_0_0/bin/python /tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py
Process exited with 3 exit code.
a_id=8979.105985621 a_implementation=python t_tenant=dti-jupyter t_tag=tc01 t_type=KerasPipeV7d19 t_action=process p_logger=action url=http://dev-dti-app-m-03:8080 connector=null mode="thick" Action failed!
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py", line 406, in _c3_remote_bootstrap__run_c3_action
    _c3_result = _action()
  File "/tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py", line 536, in <lambda>
    action=lambda: process(this = _c3_inputs.get('this'),input = _c3_inputs.get('input'),spec = _c3_inputs.get('spec')),
  File "KerasPipeV7d19.py", line 206, in process
    model = toNativeModel(this) # Always loa

C3RuntimeException: 500 - NotClassified - c3.love.util.OsUtil_err2 [381.5997]
message: "Error executing command: /usr/local/share/c3/condaEnvs/dti-jupyter/tc01/py-tensorflow_3_0_0/bin/python /tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py
Process exited with 3 exit code.
a_id=8979.105985621 a_implementation=python t_tenant=dti-jupyter t_tag=tc01 t_type=KerasPipeV7d19 t_action=process p_logger=action url=http://dev-dti-app-m-03:8080 connector=null mode="thick" Action failed!
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py", line 406, in _c3_remote_bootstrap__run_c3_action
    _c3_result = _action()
  File "/tmp/pythonActionSourceCache8586122205560385437/KerasPipeV7d19_process.py", line 536, in <lambda>
    action=lambda: process(this = _c3_inputs.get('this'),input = _c3_inputs.get('input'),spec = _c3_inputs.get('spec')),
  File "KerasPipeV7d19.py", line 206, in process
    model = toNativeMode